# 深度学习实践作业 2-b
## 可视化神经网络


### 介绍

在这个训练项目（TP）中，我们将研究各种“最近”发表的技术，以研究卷积神经网络的行为。这些方法的共同原则是使用输入图像相对于输出类别的梯度。

更正式地说，神经网络是一个数学函数 $f$，对于给定的输入图像，为每个类别关联一个分数向量 $\bar{y}$：

$$ f: \mathbb{R}^{h \times w \times c} \rightarrow \mathbb{R}^K $$

$$ x \mapsto \tilde{y} $$

我们通常对这个分数向量应用 $softmax$ 激活函数，以获得不同类别的概率分布 $\hat{y} = \text{SoftMax}(\tilde{y})$。

在这个训练项目中，目标不是修改/训练网络，而是使用一个预训练的网络，冻结其权重，并通过修改输入图像来间接研究网络的行为。首先，我们将使用Iandola等人（2016年）的SqueezeNet网络，该网络在ImageNet上训练，设计得非常紧凑且高效，同时允许获得良好的结果。

我们将看到三种方法来研究函数 $f$，通过使用分数 $\tilde{y}_i$ 相对于输入图像 $x$ 的梯度 $\frac{\partial \tilde{y}_i}{\partial x}$：

- 注意力图（Saliency Map）：基于Simonyan等人（2014年）的论文，可视化每个像素对正确类别分数的影响。

- 对抗样本（欺骗样本）：对图像进行微小的修改，这些修改对人类来说是不可察觉的，但却能导致图像被错误分类，基于Szegedy等人（2014年）的论文。

- 类别可视化（类似深度梦境）：生成一个对应于某个类别的图像，以可视化网络检测到的模式类型，基于Simonyan等人（2014年）和Yosinski等人（2015年）的论文。


### 第1节 – 显著图（Saliency Map）

显著图是一种显示图像中重要区域的图。对于研究卷积神经网络（CNN）来说，这种图对应于对正确分类预测影响最大的像素。

为了生成显著图，**Simonyan等人（2014）**提出在某个类别为$ i $的图像$ x $的邻域内，用线性函数来近似神经网络$ f $的输出：  
$$
\tilde{y}_i = f_i(x) \approx w_i^T x + b_i \quad \text{其中} \quad w_i = \frac{\partial \tilde{y}_i}{\partial x}
$$
需要注意的是，在实际操作中，图像$ x $是一个三维张量，$ w_i $同样如此。$ w_i $包含了输入图像中每个像素对应类别分数的梯度。

为了生成热力图，**Simonyan等人（2014）**建议对梯度$ w_i $的绝对值，并对每个像素的三个RGB通道取最大值，从而生成一个二维矩阵。

---

### 问题：
1. **展示并解释所获得的结果。**  
   展示显著图，并解释哪些像素对分类决策的影响最大。解释应包括高亮区域与分类结果的关联。

2. **讨论该技术的局限性。**  
   - 该方法只能反映模型对单张输入图像的反应，缺乏对整体模型行为的全局理解。  
   - 显著图可能对噪声敏感，无法准确捕捉模型的长期依赖关系。  
   - 只关注输入的梯度信息，可能会忽略更高层次的特征。

3. **该技术是否可用于解释网络之外的其他用途？**  
   可以。显著图技术也可用于其他用途，例如：
   - **弱监督目标检测**：高亮区域可用于定位目标。  
   - **模型鲁棒性分析**：分析模型对输入扰动的敏感性。  
   - **对抗样本生成**：基于显著图修改输入，提高模型误分类的概率。

4. **使用不同的网络（如VGG16）进行测试并评论。**  
   测试VGG16网络生成显著图。对比不同网络的显著图结果，分析它们在特征提取方面的差异。可以讨论更深层网络如何捕捉更抽象的特征，以及显著图在不同层次上如何变化。

### 第2节 – 对抗样本（Adversarial Examples）

对抗样本（又称欺骗样本）是近年来用于分析神经网络的一种研究方法。它通过在不显著改变图像的前提下，使神经网络对其进行错误分类。  
这一现象最早由**Szegedy等人（2014）**提出，他们展示了仅需对图像进行微小修改，就可以显著改变神经网络的预测。随后，**Goodfellow等人（2014）**和其他研究者对这一方法进行了深入研究。

**注意**：这种神经网络分析方法不应与**生成对抗网络（GANs）**混淆。尽管两者在概念上有些相似，但它们属于不同的研究领域，对抗样本主要涉及攻击和欺骗现有模型，而GAN主要用于生成数据。


### 方法概述：
有多种生成对抗样本的方法，这里介绍一种非常简单但有效的方式。

假设图像 $ x $ 被正确分类为类别 $ i $，我们的目标是通过修改图像（而非修改网络）使其被错误分类为类别 $ j $。

为此，我们对类别 $ j $ 的得分 $ \tilde{y}_j $ 相对于输入图像 $ x $ 进行**梯度反向传播**：  
$$
g = \frac{\partial \tilde{y}_j}{\partial x}
$$

接下来，我们将按照以下规则逐步修改图像 $ x $：

$$
x \leftarrow x + \eta \frac{g}{||g||_2}
$$

其中，$ \eta $ 是学习率，$ g $ 是梯度，$ ||g||_2 $ 是梯度的二范数。  
我们将该规则在选择的图像上迭代应用，直到预测的类别变为 $ j $。

---

### 问题：

5. **展示并解释所获得的结果。**  
   展示通过该方法生成的对抗样本，并解释这些样本如何使神经网络的预测类别发生变化。你可以讨论对比原始图像和经过修改后的图像，以及它们在视觉上的差异。

6. **在实践中，使用卷积神经网络时，这种方法可能会带来什么后果？**  
   在卷积神经网络（CNN）中，使用这种方法可能导致以下后果：
   - **模型脆弱性**：对抗样本使得CNN变得容易受到攻击，即使是对训练数据分布上没有显著差异的样本，也可能导致模型做出错误的预测。
   - **泛化能力下降**：网络可能会过拟合于特定类型的输入，导致它无法有效处理实际世界中的扰动。
   - **安全性问题**：如果对抗攻击能够被广泛应用，可能对使用CNN的各种应用（如自动驾驶、金融检测等）产生安全隐患。

7. **讨论这种简单构建对抗图像方法的局限性。你能提出一些替代或改进的方式吗？（可以基于近期的研究）**  
   这种方法的局限性包括：
   - **单一梯度信息**：该方法只依赖于梯度反向传播来调整图像，但这可能不会产生非常强的攻击效果，特别是在图像内容变化较少时。
   - **可见性问题**：对抗样本的修改有时可能对人眼不可察觉，但却能够有效改变模型的分类，这也可能使其更容易被防御。
   - **过度依赖学习率**：选择合适的学习率（$ \eta $）对于攻击的效果非常关键。如果学习率设置不当，可能会导致效果不明显或修改过大，导致图像失真。

   替代或改进的方法包括：
   - **快速梯度符号法（FGSM）**：Goodfellow等人提出的FGSM是一种基于梯度的对抗样本生成方法，它比上面提到的方式更加高效且稳定。
   - **目标导向的攻击**：通过优化目标损失函数，而不仅仅是依赖于梯度，可以生成更加精准的对抗样本。
   - **生成对抗网络（GANs）**：使用GAN生成对抗样本，生成网络可以学习到攻击模型的规律，从而生成更加逼真的对抗样本。
   - **迭代方法（如迭代FGSM）**：通过多次迭代逐步生成对抗样本，而不仅仅依赖于一次更新，以提高攻击的成功率。

这些方法都针对对抗样本生成过程中的不同问题进行改进，使得对抗攻击更加强大和有效。

### 第3节 – 类别可视化（Class Visualization）

为了可视化可能导致特定类别预测的模式类型，从而间接分析网络检测到的特征，**Simonyan等人（2014）**提出了一种简单的方法，随后被**Yosinski等人（2015）**进一步发展。

该方法的原理相对简单：考虑一个初始图像 $ x $（例如随机噪声），我们希望修改该图像，使其对类别 $ i $ 的得分最大化。为了提高结果的质量，建议使用一定数量的正则化技术。首先，我们将图像的 $ L_2 $ 范数作为正则化损失添加进去。此外，我们将定期对正在修改的图像应用隐式正则化，使用平移和模糊操作（这些操作已在笔记本中实现）。

形式上，对于我们希望修改的图像 $ x $ 来将其分类为类别 $ i $，我们最大化以下损失函数：
$$
L = \tilde{y}_i - \lambda ||x||_2^2
$$
然后，我们按照以下规则逐步修改图像 $ x $：
$$
x \leftarrow x + \eta \nabla_x L
$$
其中，$ \eta $ 是学习率。

---

### 问题：

8. **展示并解释所获得的结果。**  
   展示通过该方法生成的类别可视化图像，并解释这些图像反映了网络对特定类别的识别模式。分析图像中哪些区域或特征对分类起到了决定性作用。

9. **尝试改变迭代次数、学习率以及正则化权重。**  
   通过改变这些超参数，观察它们对结果的影响。讨论如何调整这些参数以优化类别可视化的效果。

10. **尝试使用ImageNet中的一张图片作为源图像，而不是随机图像（参数`init_img`）。你可以使用实际类别作为目标类别。对此进行评论。**  
    使用真实的图像作为源图像并将其分类为目标类别，可以让你更好地理解模型如何处理实际数据。这种做法有助于观察模型在真实世界样本上的表现，并揭示模型如何利用已有知识来识别目标类别。

11. **使用另一个网络（例如VGG16）进行测试，并对结果进行评论。**  
    通过使用不同的网络（如VGG16）进行相同的类别可视化任务，比较不同网络在类别可视化上的表现差异。你可以讨论不同网络架构在提取特征时的差异，以及这些差异如何影响类别可视化的结果。